In [13]:
# Setup: Add project root to Python path and change working directory
import sys
import os
from pathlib import Path

# If running from notebooks folder, go up one level to project root
if Path.cwd().name == 'notebooks':
    project_root = Path.cwd().parent
else:
    project_root = Path.cwd()

# Add to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Change working directory to project root (required for database path)
os.chdir(project_root)

print(f"Project root: {project_root}")
print(f"Working directory: {os.getcwd()}")

# Check for incomplete weeks
from src.models import SessionLocal, RSWeekly, GICSSubIndustry
from sqlalchemy import func

db = SessionLocal()
total = db.query(GICSSubIndustry).count()
print(f"Total sub-industries: {total}\n")

incomplete_weeks = []
for week, count in db.query(RSWeekly.week_end_date, func.count(RSWeekly.id)).group_by(RSWeekly.week_end_date).order_by(RSWeekly.week_end_date.desc()).all():
    if count < total:
        print(f'{week}: {count}/{total} (missing {total-count})')
        incomplete_weeks.append(week)
    else:
        print(f'{week}: {count}/{total} ✓')

if not incomplete_weeks:
    print("\n✅ All weeks are complete!")
else:
    print(f"\n⚠️ {len(incomplete_weeks)} weeks need repopulation")
    
db.close()


Project root: /Users/agustindemarchi/Documents/Pers/Projects/RS_dashboard
Working directory: /Users/agustindemarchi/Documents/Pers/Projects/RS_dashboard
2026-01-03 17:21:57,926 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-03 17:21:57,928 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT gics_subindustry.code AS gics_subindustry_code, gics_subindustry.name AS gics_subindustry_name, gics_subindustry.industry_code AS gics_subindustry_industry_code, gics_subindustry.industry_name AS gics_subindustry_industry_name, gics_subindustry.industry_group_code AS gics_subindustry_industry_group_code, gics_subindustry.industry_group_name AS gics_subindustry_industry_group_name, gics_subindustry.sector_code AS gics_subindustry_sector_code, gics_subindustry.sector_name AS gics_subindustry_sector_name 
FROM gics_subindustry) AS anon_1
2026-01-03 17:21:57,928 INFO sqlalchemy.engine.Engine [cached since 1186s ago] ()
Total sub-industries: 127

2026-01-03 17:21:57,937 INFO sq

In [14]:
# Repopulate incomplete weeks (or a specific week)
from src.models import SessionLocal
from src.services.aggregator import SubIndustryAggregator
from datetime import date

db = SessionLocal()
agg = SubIndustryAggregator(db)

# Option 1: Repopulate a specific week
# target_date = date(2025, 12, 26)
# records = agg.store_weekly_rs(target_date)
# print(f"Repopulated {records} records for {target_date}")

# Option 2: Repopulate all incomplete weeks from previous cell
if 'incomplete_weeks' in dir() and incomplete_weeks:
    for week in incomplete_weeks:
        records = agg.store_weekly_rs(week)
        print(f"Repopulated {records} records for {week}")
    print(f"\n✅ Finished repopulating {len(incomplete_weeks)} weeks")
else:
    print("No incomplete weeks to repopulate")

db.close()


No incomplete weeks to repopulate


In [16]:
# View the entire RSWeekly dataset
import pandas as pd
from src.models import SessionLocal, RSWeekly, GICSSubIndustry

db = SessionLocal()

# Query all RSWeekly records with sub-industry names
query = db.query(
    RSWeekly.id,
    RSWeekly.subindustry_code,
    GICSSubIndustry.name.label('subindustry_name'),
    GICSSubIndustry.sector_name,
    RSWeekly.week_end_date,
    RSWeekly.week_start_date,
    RSWeekly.rs_line,
    RSWeekly.rs_line_sma_52w,
    RSWeekly.mansfield_rs,
    RSWeekly.rs_percentile,
    RSWeekly.constituents_count
).join(
    GICSSubIndustry, RSWeekly.subindustry_code == GICSSubIndustry.code
).filter(
    # GICSSubIndustry.name.label('subindustry_name') == "Semiconductor Materials & Equipment"
    GICSSubIndustry.name.label('subindustry_name') == "Technology Hardware, Storage & Peripherals"
).order_by(RSWeekly.week_end_date.desc(), RSWeekly.rs_percentile.desc())

# Convert to DataFrame
df = pd.read_sql(query.statement, db.bind)
db.close()

print(f"Total records: {len(df)}")
print(f"Date range: {df['week_end_date'].min()} to {df['week_end_date'].max()}")
print(f"Unique weeks: {df['week_end_date'].nunique()}")
print(f"Unique sub-industries: {df['subindustry_code'].nunique()}")
print()

# Display the dataframe
df

2026-01-03 17:23:16,325 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-03 17:23:16,327 INFO sqlalchemy.engine.Engine SELECT rs_weekly.id, rs_weekly.subindustry_code, gics_subindustry.name AS subindustry_name, gics_subindustry.sector_name, rs_weekly.week_end_date, rs_weekly.week_start_date, rs_weekly.rs_line, rs_weekly.rs_line_sma_52w, rs_weekly.mansfield_rs, rs_weekly.rs_percentile, rs_weekly.constituents_count 
FROM rs_weekly JOIN gics_subindustry ON rs_weekly.subindustry_code = gics_subindustry.code 
WHERE gics_subindustry.name = ? ORDER BY rs_weekly.week_end_date DESC, rs_weekly.rs_percentile DESC
2026-01-03 17:23:16,327 INFO sqlalchemy.engine.Engine [cached since 582.4s ago] ('Technology Hardware, Storage & Peripherals',)
2026-01-03 17:23:16,331 INFO sqlalchemy.engine.Engine ROLLBACK
Total records: 18
Date range: 2025-09-05 to 2026-01-02
Unique weeks: 18
Unique sub-industries: 1



,id,subindustry_code,subindustry_name,sector_name,week_end_date,week_start_date,rs_line,rs_line_sma_52w,mansfield_rs,rs_percentile,constituents_count
0,2252,45460297,"Technology Hardware, Storage & Peripherals",Information Technology,2026-01-02,2025-12-29,21.525719,None,None,None,b'\t\x00\x00\x00\x00\x00\x00\x00'
1,93,45460297,"Technology Hardware, Storage & Peripherals",Information Technology,2025-12-26,2025-12-22,21.075073,None,None,None,b'\t\x00\x00\x00\x00\x00\x00\x00'
2,220,45460297,"Technology Hardware, Storage & Peripherals",Information Technology,2025-12-19,2025-12-15,21.306097,None,None,None,b'\t\x00\x00\x00\x00\x00\x00\x00'
3,347,45460297,"Technology Hardware, Storage & Peripherals",Information Technology,2025-12-12,2025-12-08,20.826644,None,None,None,b'\t\x00\x00\x00\x00\x00\x00\x00'
4,474,45460297,"Technology Hardware, Storage & Peripherals",Information Technology,2025-12-05,2025-12-01,21.028119,None,None,None,b'\t\x00\x00\x00\x00\x00\x00\x00'
5,601,45460297,"Technology Hardware, Storage & Peripherals",Information Technology,2025-11-28,2025-11-24,20.643285,None,None,None,b'\t\x00\x00\x00\x00\x00\x00\x00'
6,728,45460297,"Technology Hardware, Storage & Peripherals",Information Technology,2025-11-21,2025-11-17,19.510010,None,None,None,b'\t\x00\x00\x00\x00\x00\x00\x00'
7,855,45460297,"Technology Hardware, Storage & Peripherals",Information Technology,2025-11-14,2025-11-10,21.035759,None,None,None,b'\t\x00\x00\x00\x00\x00\x00\x00'
8,982,45460297,"Technology Hardware, Storage & Peripherals",Information Technology,2025-11-07,2025-11-03,21.555587,None,None,None,b'\t\x00\x00\x00\x00\x00\x00\x00'
9,1109,45460297,"Technology Hardware, Storage & Peripherals",Information Technology,2025-10-31,2025-10-27,20.555764,None,None,None,b'\t\x00\x00\x00\x00\x00\x00\x00'


In [8]:
# View a specific GICSSubIndustry by code
import pandas as pd
from src.models import SessionLocal, GICSSubIndustry

db = SessionLocal()

# Query specific GICSSubIndustry record
query = db.query(
    GICSSubIndustry.code,
    GICSSubIndustry.name,
    GICSSubIndustry.industry_code,
    GICSSubIndustry.industry_name,
    GICSSubIndustry.industry_group_code,
    GICSSubIndustry.industry_group_name,
    GICSSubIndustry.sector_code,
    GICSSubIndustry.sector_name
).filter(
    GICSSubIndustry.code == "45460297"
)

# Convert to DataFrame
df_gics = pd.read_sql(query.statement, db.bind)
db.close()

print(f"Records found: {len(df_gics)}")
print()

# Display the dataframe
df_gics


2026-01-03 17:10:47,621 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-03 17:10:47,622 INFO sqlalchemy.engine.Engine SELECT gics_subindustry.code, gics_subindustry.name, gics_subindustry.industry_code, gics_subindustry.industry_name, gics_subindustry.industry_group_code, gics_subindustry.industry_group_name, gics_subindustry.sector_code, gics_subindustry.sector_name 
FROM gics_subindustry 
WHERE gics_subindustry.code = ?
2026-01-03 17:10:47,624 INFO sqlalchemy.engine.Engine [generated in 0.00227s] ('45460297',)
2026-01-03 17:10:47,629 INFO sqlalchemy.engine.Engine ROLLBACK
Records found: 1



,code,name,industry_code,industry_name,industry_group_code,industry_group_name,sector_code,sector_name
0,45460297,"Technology Hardware, Storage & Peripherals",454602,"Technology Hardware, Storage & Peripherals",4546,"Technology Hardware, Storage & Peripherals",45,Information Technology


In [12]:
# Investigate the NaN issue for Semiconductor Materials & Equipment
import pandas as pd
from src.models import SessionLocal, Stock, StockPrice, GICSSubIndustry
from sqlalchemy import func

db = SessionLocal()

# Find stocks in the problematic sub-industry
subindustry_code = "45537154"  # Semiconductor Materials & Equipment

stocks = db.query(Stock).filter(
    Stock.gics_subindustry_code == subindustry_code,
    Stock.is_active == True
).all()

print(f"Sub-industry: Semiconductor Materials & Equipment ({subindustry_code})")
print(f"Number of stocks: {len(stocks)}")
print(f"Stocks: {[s.ticker for s in stocks]}")
print()

# Check price data date range for these stocks
for stock in stocks:
    price_range = db.query(
        func.min(StockPrice.date),
        func.max(StockPrice.date),
        func.count(StockPrice.id)
    ).filter(StockPrice.ticker == stock.ticker).first()
    
    print(f"{stock.ticker}: {price_range[0]} to {price_range[1]} ({price_range[2]} records)")

# Also check SPY (benchmark) data range
spy_range = db.query(
    func.min(StockPrice.date),
    func.max(StockPrice.date),
    func.count(StockPrice.id)
).filter(StockPrice.ticker == "SPY").first()

print(f"\nSPY (benchmark): {spy_range[0]} to {spy_range[1]} ({spy_range[2]} records)")

# Calculate how many weeks of data we have
if spy_range[0] and spy_range[1]:
    from datetime import datetime
    weeks = (spy_range[1] - spy_range[0]).days / 7
    print(f"Total weeks of SPY data: {weeks:.1f} weeks")
    print(f"52-week SMA requires: 52 weeks (~260 trading days)")

db.close()


2026-01-03 17:16:34,793 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-03 17:16:34,797 INFO sqlalchemy.engine.Engine SELECT stock.ticker AS stock_ticker, stock.name AS stock_name, stock.gics_subindustry_code AS stock_gics_subindustry_code, stock.market_cap AS stock_market_cap, stock.is_active AS stock_is_active, stock.created_at AS stock_created_at, stock.updated_at AS stock_updated_at 
FROM stock 
WHERE stock.gics_subindustry_code = ? AND stock.is_active = 1
2026-01-03 17:16:34,797 INFO sqlalchemy.engine.Engine [generated in 0.00073s] ('45537154',)
Sub-industry: Semiconductor Materials & Equipment (45537154)
Number of stocks: 5
Stocks: ['AMAT', 'KLAC', 'LRCX', 'Q', 'TER']

2026-01-03 17:16:34,806 INFO sqlalchemy.engine.Engine SELECT min(stock_price.date) AS min_1, max(stock_price.date) AS max_1, count(stock_price.id) AS count_1 
FROM stock_price 
WHERE stock_price.ticker = ?
 LIMIT ? OFFSET ?
2026-01-03 17:16:34,806 INFO sqlalchemy.engine.Engine [generated in 0.00060s] ('AMAT'